In [ ]:
import numpy as np
import keras
import scipy.io as sio
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from resnet50_utils import *
from aid_h5_utils import *
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, GlobalAveragePooling2D

import keras.backend as K
K.set_image_data_format('channels_last')

In [ ]:
X_train, Y_train, X_test, Y_test, classes = load_dataset()

In [ ]:
model = ResNet50(input_shape = (224, 224, 3), classes = 30)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])

In [ ]:
model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', by_name=True)

In [ ]:
# Save file 1 - best model
checkpoint = ModelCheckpoint(filepath='resnet50_best_model.h5',monitor='val_acc',mode='auto' ,save_best_only='True')
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
callback_lists=[checkpoint,reducelr]

In [ ]:
history = model.fit(X_train,Y_train,batch_size=64,epochs=30,validation_data=(X_test,Y_test),callbacks=callback_lists)

In [ ]:
best_model = load_model('resnet50_dropout_best_model.h5')

In [ ]:
pred_y = best_model.predict(X_test, verbose=1)

In [ ]:
predict_label = np.argmax(pred_y, axis=1)
true_label = np.argmax(Y_test, axis=1)

In [ ]:
# Save file 2 - consusion matrix
f = h5py.File('predict_true_label_for_confusion_matrix', 'w')
f['predict_label'] = predict_label
f['true_label'] = true_label
f.close()

In [ ]:
# Save file 3 - loss, val_loss, acc, val_acc
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = np.array(loss)
val_loss = np.array(val_loss)
acc = np.array(acc)
val_acc = np.array(val_acc)

fp = h5py.File('', 'w')
fp['loss'] = loss
fp['val_loss'] = val_loss
fp['acc'] = acc
fp['val_acc'] = val_acc
fp.close()

In [ ]:
# Save file 4 - best model weights
best_model.save_weights('resnet50_model_weights.h5')

In [ ]:
# Save file 5 - model structure
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
# Save file 6 - model summary
model.summary()